### 1- Mine pull requests and issues from projects

For each project:
* get pull requests and save into database.
* get issues and labels and save into database.

Projects Selection:
 - qtd_stars >= 100
 - forked is false
 - size  > 10000
 - ci_service is null or (ci_service like 'Travis CI' and analysis_point is not null)

In [18]:
repos = getProjects()
i=0
for repo in repos:
    project = repo[0]
    i += 1
    print('\n\n\n{}/{} Project - {}\n\n\n'.format(i,len(repos),project))
    
    qPullRequests = 'https://api.github.com/repos/{}/pulls?state=all&sort=created&direction=asc&per_page=100&page='.format(project)
    minePRs(project,qPullRequests)
    setProjectPRMined(project)
    
    qIssues = 'https://api.github.com/repos/{}/issues?state=all&filter=all&sort=created&direction=asc&per_page=100&page='.format(project)
    mineIssues(project,qIssues)
    setProjectIssueMined(project)
    
    saveCreationDate(project)




1/3 Project - momotech/MLN



+ Project: momotech/MLN



2/3 Project - muzei/muzei



+ Project: muzei/muzei



3/3 Project - spatialos/gdk-for-unity



+ Project: spatialos/gdk-for-unity


### Aux Functions

In [2]:
import psycopg2
import os
import requests 
import time
from datetime import datetime, timedelta
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def connectDB():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "ci_quality",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v3")

In [4]:
def minePRs(project,url):
    page = 1
    prs= requestAPI(url+str(page))
    
    while prs is not None and len(prs) > 0:
        #processPRs(prs)
        try:
            for pr in prs:
                print('+ Project: {}    PR - {}'.format(project, pr['id']))
                storePR(project,pr)
        
        except Exception as e:
            print('\n\n==============================================================')
            print('Error while processing storePR >>> Exception: {}'.format(e)) 
            print('Project: {}    PR - {}\n\n'.format(project, url+str(page)))
        
        page += 1
        prs= requestAPI(url+str(page))

In [5]:
def mineIssues(project,url):
    page = 1
    issues= requestAPI(url+str(page))
    
    while issues is not None and len(issues) > 0:
        try:
            for issue in issues:
                print('+ Project: {}    Issue - {}'.format(project, issue['id']))
                storeIssue(project,issue)
                
                #Get and save Labels
                saveIssueLabels(project,issue['number'],issue['url'])
        
        except Exception as e:
            print('\n\n==============================================================')
            print('Error while processing storeIssue >>> Exception: {}'.format(e)) 
            print('Project: {}    PR - {}\n\n'.format(project, url+str(page)))
        
        page += 1
        issues= requestAPI(url+str(page))

In [6]:
def saveIssueLabels(repo_name, issue_number, issue_url):
    try:
        print('+ Project: {}    issue - {}'.format(repo_name, issue_number))
        
        labels = requestAPI(issue_url+'/labels')
        if labels is None: return
        
        for l in labels:
            storeLabel(l,repo_name, issue_number)
        
    except Exception as e:
        print('==============================================================')
        print('Error while processing saveIssueLabels >>> Exception: {}'.format(e)) 
        print('Project: {}    issue - {}'.format(repo_name, issue_number))

In [7]:
def saveCreationDate(repo_name):
    try:
        print('+ Project: {}'.format(repo_name))
    
        
        project = requestAPI('https://api.github.com/repos/{}'.format(repo_name))
        creation = project['created_at']
        if creation is None: return
        
        updateRepo(repo_name, creation)
        
    except Exception as e:
        print('==============================================================')
        print('Error while processing saveIssueLabels >>> Exception: {}'.format(e)) 
        print('Project: {}'.format(repo_name))

In [8]:
def storeIssue(repo_name, issue):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """INSERT INTO public.issue(
    id, issue_number, url, repo_url, labels_url, comments_url, events_url, html_url, node_id, title, state, locked, 
    comments, created_at, updated_at, closed_at, author_association, body, repo_name)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
    
    cursor.execute(query, [issue['id'],issue['number'],issue['url'],issue['repository_url'],issue['labels_url'],issue['comments_url'],
                           issue['events_url'],issue['html_url'],issue['node_id'],issue['title'],issue['state'],
                           issue['locked'],issue['comments'],issue['created_at'],issue['updated_at'],issue['closed_at'],
                           issue['author_association'],issue['body'],repo_name])
    connection.commit()
    
    cursor.close()
    connection.close()

In [9]:
def storeLabel(label,repo_name, issue_number):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """SELECT idlabel FROM label WHERE idlabel = %s"""
    cursor.execute(query, [label['id']])
    row = cursor.fetchone()
    
    if row is None: 
        query = """INSERT INTO Label (idlabel,name,label_default,description,url) VALUES(%s,%s,%s,%s,%s)"""
        
        cursor.execute(query, [label['id'],label['name'],label['default'],label['description'],label['url']])
        connection.commit()
    
    query = """INSERT INTO issueLabel (issue_number,repo_name,idlabel) VALUES(%s,%s,%s)"""
    cursor.execute(query, [issue_number,repo_name,label['id']])
    connection.commit()
    
    cursor.close()
    connection.close()

In [10]:
def storePR(repo_name, pr):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """INSERT INTO public.pullrequests(
    id, author_association, closed_at, comments_url, commits_url, created_at, merge_commit_sha, merged_at, pr_number, 
    review_comment_url, review_comments_url, state, statuses_url, title, updated_at, project_name, id_user, mined)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    
    cursor.execute(query, [pr['id'],pr['author_association'],pr['closed_at'],pr['comments_url'],pr['commits_url'],
                           pr['created_at'],pr['merge_commit_sha'],pr['merged_at'],pr['number'],pr['review_comments_url'],
                           pr['review_comments_url'],pr['state'],pr['statuses_url'],pr['title'],pr['updated_at'],
                           repo_name,pr['user']['id'],False])
    connection.commit()
    
    cursor.close()
    connection.close()

In [11]:
def updateRepo(repo_name, creation):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE projects
        set created = %s
        WHERE REPO_NAME = %s;"""
    
    cursor.execute(query, [creation,repo_name])
    connection.commit()
    
    cursor.close()
    connection.close()

In [12]:
tokens =['token 80665cc2fa2fcf9f6368ac32014eda5bf5083bd3', 'token 1f3a6e7d066f937c163900db140e4506a2f3dd87', 'token 1f5163feb5b40cf70542f3a4360b957d9d6a668a']
i_token =0


def requestAPI(URL):
    # api-endpoint 
    global i_token
    
    if (len(tokens)-1) > i_token:
        i_token += 1
    else:
        i_token = 0
        
    r = requests.get(url = URL, headers={'Authorization': tokens[i_token],'Accept':'application/vnd.github.cloak-preview'}).json()
    if len(r) > 0:
        try:
            #print('keys: {}'.format(r.keys()))
            
            if isinstance(r, dict) and 'message' in r.keys():
                print('{} -- {}'.format(datetime.now().strftime("%H:%M:%S"),r['message']))
                if 'API rate limit exceeded' in r['message']:
                    time.sleep(600)
                    requestAPI(URL)
                else:
                    return None

            return r
        except Exception as e:
            print('\n Erro no request get: {}'.format(e))
            print(r)
    else:
        return None

In [17]:
def getProjects():
    query = """select repo_name from projects
            where qtd_stars >= 100 and forked is false and size  > 10000
            and (ci_service is null or (ci_service like 'Travis CI' and analysis_point is not null))
            and prs_mined is false
            ORDER BY repo_name """
    
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [14]:
def setProjectPRMined(repo_name):
    query = """UPDATE  projects 
        set prs_mined = True
        WHERE repo_name like %s"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [repo_name])
    connection.commit()
    cursor.close()
    connection.close()

In [15]:
def setProjectIssueMined(repo_name):
    query = """UPDATE  projects 
        set issues_mined = True
        WHERE repo_name like %s"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query, [repo_name])
    connection.commit()
    cursor.close()
    connection.close()